<a href="https://colab.research.google.com/github/thiagodepaulo/nlp/blob/enap/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os.path
import glob
import codecs
import nltk
import re
import spacy
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import unicodedata
from sklearn.feature_extraction.text import TfidfTransformer
import pandas as pd

In [2]:
corpus = ["""Naruto (ナルト?) é uma série de mangá escrita e ilustrada por Masashi Kishimoto,
que conta a história de Naruto Uzumaki, um jovem ninja que constantemente procura por
reconhecimento e sonha em se tornar Hokage, o ninja líder de sua vila.
A história é dividida em duas partes, a primeira parte se passa nos anos da pré-adolescência de Naruto,
e a segunda parte se passa em sua adolescência. A série é baseada em dois mangás one-shots de Kishimoto:
Karakuri (1995), e que por causa dele Kishimoto ganhou uma menção honrosa no prêmio Hop Step da Shueisha
no ano seguinte, e Naruto (1997). A editora Panini Comics licenciou e publicou o mangá em
três versões no Brasil, e em Portugal é a editora Devir Manga quem licencia e publica o mangá.
A história de Naruto continua com o seu filho, Boruto Uzumaki em Boruto: Naruto Next Generations.""",

"""Serial Experiments Lain (シリアルエクスペリメンツレイン Shiriaru Ekusuperimentsu Rein?) é uma série anime
lançada pelo estúdio Triangle Staff, dirigida por Ryutaro Nakamura, criação de cenários por
Chiaki J. Konaka e desenhada por Yoshitoshi ABe. A série foi transmitida pela TV Tokyo de Julho a
Setembro de 1998 e teve um jogo para PlayStation lançado no mesmo ano.""",

"""Fullmetal Alchemist (鋼の錬金術師 Hagane no Renkinjutsushi?, lit. "Alquimista de Aço")
é um anime baseado no mangá shōnen escrito e ilustrado por Hiromu Arakawa. O mangá foi
serializado na revista mensal japonesa Monthly Shōnen Gangan entre agosto de 2001 e junho de 2010,
com os seus 108 capítulos individuais compilados em 27 volumes em formato tankōbon e publicados pela
editora Square Enix. O mundo de Fullmetal Alchemist é baseado no período após a Revolução Industrial
Europeia. Situado em um universo ficcional em que a alquimia é uma das mais avançadas técnicas científicas
conhecidas pelo homem, a história centra-se nos irmãos Edward Elric e Alphonse Elric, que estão procurando
a pedra filosofal para restaurar seus corpos após uma desastrosa tentativa de trazer a mãe falecida de volta à
vida através da alquimia."""
]

In [3]:
from transformers import AutoTokenizer

model = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model)
enc_text = tokenizer(corpus)
print(enc_text)

{'input_ids': [[101, 6583, 22134, 2080, 1006, 1715, 30259, 30240, 1029, 1007, 1041, 8529, 2050, 8668, 2139, 8952, 9686, 26775, 6590, 1041, 6335, 19966, 28510, 18499, 16137, 12914, 11382, 6182, 15319, 1010, 10861, 9530, 2696, 1037, 19891, 2139, 6583, 22134, 2080, 1057, 9759, 2863, 3211, 1010, 8529, 8183, 3726, 2213, 14104, 10861, 5377, 13665, 2063, 4013, 10841, 2527, 18499, 28667, 2239, 5369, 6895, 23065, 1041, 2365, 3270, 7861, 7367, 7950, 2906, 7570, 2912, 3351, 1010, 1051, 14104, 11876, 2121, 2139, 10514, 2050, 23840, 1012, 1037, 19891, 1041, 4487, 17258, 8524, 7861, 4241, 3022, 2112, 2229, 1010, 1037, 3539, 7895, 2112, 2063, 7367, 3413, 2050, 16839, 2019, 2891, 4830, 3653, 1011, 4748, 29111, 27524, 7405, 2139, 6583, 22134, 2080, 1010, 1041, 1037, 19071, 2112, 2063, 7367, 3413, 2050, 7861, 10514, 2050, 4748, 29111, 27524, 7405, 1012, 1037, 8668, 1041, 2918, 8447, 7861, 9193, 2015, 8952, 2015, 2028, 1011, 7171, 2139, 11382, 6182, 15319, 1024, 13173, 18569, 2072, 1006, 2786, 1007, 1010

In [6]:
tokens = tokenizer.convert_ids_to_tokens(enc_text.input_ids[0])
print(tokens)

['[CLS]', 'na', '##rut', '##o', '(', 'ナ', '##ル', '##ト', '?', ')', 'e', 'um', '##a', 'serie', 'de', 'manga', 'es', '##cr', '##ita', 'e', 'il', '##ust', '##rada', 'por', 'mas', '##ashi', 'ki', '##shi', '##moto', ',', 'que', 'con', '##ta', 'a', 'historia', 'de', 'na', '##rut', '##o', 'u', '##zu', '##ma', '##ki', ',', 'um', 'jo', '##ve', '##m', 'ninja', 'que', 'constant', '##ement', '##e', 'pro', '##cu', '##ra', 'por', 'rec', '##on', '##he', '##ci', '##mento', 'e', 'son', '##ha', 'em', 'se', 'torn', '##ar', 'ho', '##ka', '##ge', ',', 'o', 'ninja', 'lid', '##er', 'de', 'su', '##a', 'vila', '.', 'a', 'historia', 'e', 'di', '##vid', '##ida', 'em', 'du', '##as', 'part', '##es', ',', 'a', 'prime', '##ira', 'part', '##e', 'se', 'pass', '##a', 'nos', 'an', '##os', 'da', 'pre', '-', 'ad', '##oles', '##cen', '##cia', 'de', 'na', '##rut', '##o', ',', 'e', 'a', 'segunda', 'part', '##e', 'se', 'pass', '##a', 'em', 'su', '##a', 'ad', '##oles', '##cen', '##cia', '.', 'a', 'serie', 'e', 'base', '##ada', 

In [7]:
print(tokenizer.convert_tokens_to_string(tokens))

[CLS] naruto ( ナルト? ) e uma serie de manga escrita e ilustrada por masashi kishimoto, que conta a historia de naruto uzumaki, um jovem ninja que constantemente procura por reconhecimento e sonha em se tornar hokage, o ninja lider de sua vila. a historia e dividida em duas partes, a primeira parte se passa nos anos da pre - adolescencia de naruto, e a segunda parte se passa em sua adolescencia. a serie e baseada em dois mangas one - shots de kishimoto : karakuri ( 1995 ), e que por causa dele kishimoto ganhou uma mencao honrosa no premio hop step da shueisha no ano seguinte, e naruto ( 1997 ). a editora panini comics licenciou e publicou o manga em tres versoes no brasil, e em portugal e a editora devir manga quem licencia e publica o manga. a historia de naruto continua com o seu filho, boruto uzumaki em boruto : naruto next generations. [SEP]


# Pré-processamento

limpando o texto

In [ ]:
corpus_clean1 = [ unicodedata.normalize('NFKD', s).encode('ISO-8859-1', 'ignore').decode('ISO-8859-1')
                 for s in corpus]

In [ ]:
single_char = 'ê'
multiple_chars = '\N{LATIN SMALL LETTER E}\N{COMBINING CIRCUMFLEX ACCENT}'

In [ ]:
unicodedata.normalize('NFKD', corpus_clean1[0]).encode('ISO-8859-1', 'ignore')

b'Naruto (?) e uma serie de manga escrita e ilustrada por Masashi Kishimoto, \nque conta a historia de Naruto Uzumaki, um jovem ninja que constantemente procura por \nreconhecimento e sonha em se tornar Hokage, o ninja lider de sua vila. \nA historia e dividida em duas partes, a primeira parte se passa nos anos da pre-adolescencia de Naruto, \ne a segunda parte se passa em sua adolescencia. A serie e baseada em dois mangas one-shots de Kishimoto: \nKarakuri (1995), e que por causa dele Kishimoto ganhou uma mencao honrosa no premio Hop Step da Shueisha \nno ano seguinte, e Naruto (1997). A editora Panini Comics licenciou e publicou o manga em \ntres versoes no Brasil, e em Portugal e a editora Devir Manga quem licencia e publica o manga. \nA historia de Naruto continua com o seu filho, Boruto Uzumaki em Boruto: Naruto Next Generations.'

In [ ]:
# stop words
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
stop_list = nltk.corpus.stopwords.words('portuguese')

In [ ]:
corpus_tokens = [nltk.word_tokenize(doc) for doc in corpus_clean]

In [ ]:
# Stemming

corpus_tokens2 = [nltk.word_tokenize(doc) for doc in corpus_clean]
ps = SnowballStemmer('portuguese')
def do_stem(tokens):
    return [ps.stem(word) for word in tokens]

corpus_stem = [' '.join(do_stem(doc)) for doc in corpus_tokens2]

In [ ]:
ps.stem('falando'), ps.stem('falado')

('fal', 'fal')

In [ ]:
# Lematizador
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 24.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
sp = spacy.load("pt_core_news_sm")

In [ ]:
doc = sp(corpus[0])
' '.join([token.lemma_ for token in doc])

'Naruto ( ナルト ? ) ser um série de mangá escrever e ilustrar por Masashi Kishimoto , \n que contar o história de Naruto Uzumaki , um jovem ninja que constantemente procurar por \n reconhecimento e sonhar em se tornar Hokage , o ninja líder de seu vila . \n o história ser divider em dois parte , o primeiro parte se passar em o ano de o pré-adolescência de Naruto , \n e o segundo parte se passar em seu adolescência . o série ser basear em dois mangás one-shots de Kishimoto : \n Karakuri ( 1995 ) , e que por causa de ele Kishimoto ganhar um menção honroso em o prêmio Hop Step de o Shueisha \n em o ano seguinte , e Naruto ( 1997 ) . o editora Panini Comics licenciar e publicar o mangá em \n três versão em o Brasil , e em Portugal ser o editora Devir Manga quem licencia e publicar o mangá . \n o história de Naruto continuar com o seu filho , Boruto Uzumaki em Boruto : Naruto Next Generations .'

In [ ]:
# Criando Matrizes

In [ ]:
from scipy.sparse import csr_matrix
import numpy as np

indptr = np.array([0,2,3,6])
indices = np.array([0,2,2,0,1,2])
data = np.array([1,2,3,4,5,6])
S = csr_matrix( (data,indices,indptr))
print(S)

  (0, 0)	1
  (0, 2)	2
  (1, 2)	3
  (2, 0)	4
  (2, 1)	5
  (2, 2)	6


In [ ]:
S.todense()

matrix([[1, 0, 2],
        [0, 0, 3],
        [4, 5, 6]])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

count_vect = CountVectorizer()
count_vect.fit(corpus)
X = count_vect.transform(corpus)
X.shape

(3, 194)

In [ ]:
names = count_vect.get_feature_names_out()

In [ ]:
print( [ (names[w],w) for w in X.getrow(0).nonzero()[1]] )

[('1995', 1), ('1997', 2), ('adolescência', 8), ('ano', 15), ('anos', 16), ('baseada', 22), ('boruto', 24), ('brasil', 25), ('causa', 27), ('com', 32), ('comics', 33), ('constantemente', 36), ('conta', 37), ('continua', 38), ('da', 41), ('de', 43), ('dele', 44), ('devir', 47), ('dividida', 49), ('dois', 50), ('duas', 51), ('editora', 52), ('em', 56), ('escrita', 59), ('filho', 67), ('ganhou', 73), ('generations', 74), ('história', 77), ('hokage', 78), ('honrosa', 80), ('hop', 81), ('ilustrada', 82), ('jovem', 89), ('karakuri', 92), ('kishimoto', 93), ('licencia', 98), ('licenciou', 99), ('líder', 101), ('manga', 103), ('mangá', 104), ('mangás', 105), ('masashi', 106), ('menção', 108), ('naruto', 115), ('next', 116), ('ninja', 117), ('no', 118), ('nos', 119), ('one', 120), ('panini', 122), ('parte', 124), ('partes', 125), ('passa', 126), ('por', 132), ('portugal', 133), ('primeira', 134), ('procura', 135), ('pré', 137), ('prêmio', 138), ('publica', 139), ('publicou', 141), ('que', 142),

In [ ]:
hash_vect = HashingVectorizer(n_features=10000)
X3 = hash_vect.fit_transform(corpus)
X3.shape

(3, 10000)